<a href="https://colab.research.google.com/github/ImpulsoGov/Analises-Publicas-Saude/blob/main/Notebooks/202205_Atividade_Coletiva_Reunioes_Territorios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise do perfil dos municipios e de suas atividades quanto a monitoramento do territorio

Este notebook consome dados públicos do  Sistema de Informação em Saúde para a Atenção Básica (SISAB) e de informações auxiliares como IBGE, com intenção de determinar o perfil dos municipios e de suas atividades quanto a monitoramento do territorio.

As análises estão limitadas a datas a partir de abril de 2013. 


### Fonte dos dados
Os dados são provenientes do site do SISAB, disponível em: https://sisab.saude.gov.br/paginas/acessoRestrito/relatorio/federal/saude/RelSauAtivColetiva.xhtml. 
O download e processamento dos dados utiliza requisição http feitas internamente como segue abaixo.

## **Preparação do ambiente**

In [1]:
import pandas as pd
import math
import plotly.express as px
import plotly.graph_objects as go

## **Requisição e tratamento dos dados**

In [ ]:
# dados = pd.read_csv("/content/RelAtvColetivaMB (27).csv", header=None, encoding = 'unicode_escape', engine= 'python')
# dados = pd.read_csv(StringIO("/content/RelAtvColetivaMB (27).csv"), header=None)

for i in range(12, 63):
  filename = "/content/RelAtvColetivaMB ("+str(i)+").csv"
  dados = pd.read_csv(filename, header=None, on_bad_lines='skip', engine='python', encoding='unicode_escape')
  nomearquivofinal = str(dados.loc[3][0])[14:-6]+"_"+str(dados.loc[3][0])[18:-1]+"_RelAtvColetivaMB.csv"
  dados.columns = dados.loc[6]
  dados = dados.iloc[7:-4]
  dados.to_csv(nomearquivofinal, encoding='utf-8')

In [ ]:
meses=[
    ('JAN','1'),
    ('FEV','2'),
    ('MAR','3'),
    ('ABR','4'),
    ('MAI','5'),
    ('JUN','6'),
    ('JUL','7'),
    ('AGO','8'),
    ('SET','9'),
    ('OUT','10'),
    ('NOV','11'),
    ('DEZ','12'),
]      

anos=['2018', '2019', '2020', '2021']
# anos=['2022']

dados_juntados = pd.DataFrame(columns=[',Uf', 'Ibge', 'Municipio', 'codigo', 'Questões administrativas / fun', 'Processos de trabalho', 'Diagnóstico do território / mo', 'Planejamento / monitoramento d', 'Discussão de caso / projeto te', 'Educação permanente', 'Outros'])

for ano in anos:
  for j in range(len(meses)):
      filename = meses[j][0]+"_"+ano+"_RelAtvColetivaMB.csv"
      dados_juntados_parcial = pd.DataFrame(columns=[',Uf', 'Ibge', 'Municipio', 'codigo', 'Questões administrativas / fun', 'Processos de trabalho', 'Diagnóstico do território / mo', 'Planejamento / monitoramento d', 'Discussão de caso / projeto te', 'Educação permanente', 'Outros'])
      dados = pd.read_csv(filename, sep=';')
      codigo = ano+".M"+meses[j][1]
      dados_juntados_parcial = dados
      dados_juntados_parcial['codigo'] = codigo
      dados_juntados = dados_juntados.append(dados_juntados_parcial)
dados_juntados.to_csv('dadosrelativcoletiva.csv')

## **Merge**

In [2]:
periodos = pd.read_csv('periodos_202205292217.csv')
dadosrelativcoletiva = pd.read_csv('dadosrelativcoletiva.csv')
dadosrelativcoletiva2022 = pd.read_csv('dadosrelativcoletiva_2022.csv')

In [3]:
dadosrelativcoletiva = dadosrelativcoletiva.append(dadosrelativcoletiva2022)

In [4]:
dadosrelativcoletiva = dadosrelativcoletiva.merge(periodos, on='codigo')

In [5]:
# dadosrelativcoletiva.info()
dadosrelativcoletiva['data_inicio'] = dadosrelativcoletiva['data_inicio'].astype('datetime64[ns]')

In [10]:
dadosrelativcoletiva.rename(columns = {'Questões administrativas / fun':'Questões administrativas / funcionamento', 
                     'Diagnóstico do território / mo':'Diagnóstico do território / monitoramento do terrotório',
                     'Planejamento / monitoramento d':'Planejamento / monitoramento das ações da equipe',
                     'Discussão de caso / projeto te':'Discussão de caso / projeto terapêutico singular'}, inplace = True)

## **Analises**

### Reuniões por Municipios

In [16]:
copiadados = dadosrelativcoletiva

In [26]:
# dadosagrupadosportema.melt(id_vars=['Questões administrativas / funcionamento', 'Processos de trabalho',
#        'Diagnóstico do território / monitoramento do terrotório', 'Planejamento / monitoramento das ações da equipe',
#        'Discussão de caso / projeto terapêutico singular', 'Educação permanente', 'Outros'], value_vars=['Ibge'])

# pd.pivot_table(dadosagrupadosportema, values=['Ibge'], index=['Questões administrativas / funcionamento', 'Processos de trabalho',
#        'Diagnóstico do território / monitoramento do terrotório', 'Planejamento / monitoramento das ações da equipe',
#        'Discussão de caso / projeto terapêutico singular', 'Educação permanente', 'Outros'],
#                     aggfunc={'Ibge': np.mean})


dadosagrupadosportema = copiadados.groupby(['Questões administrativas / funcionamento', 'Processos de trabalho',
       'Diagnóstico do território / monitoramento do terrotório', 'Planejamento / monitoramento das ações da equipe',
       'Discussão de caso / projeto terapêutico singular', 'Educação permanente', 'Outros'], as_index=False)['Ibge'].count()
dadosagrupadosportema
# dadosagrupadosportema.apply(pd.value_counts).fillna(0)

,Questões administrativas / funcionamento,Processos de trabalho,Diagnóstico do território / monitoramento do terrotório,Planejamento / monitoramento das ações da equipe,Discussão de caso / projeto terapêutico singular,Educação permanente,Outros,Ibge
0,0,0,0,0,0,0,0,59561
1,0,0,0,0,0,0,1,2732
2,0,0,0,0,0,0,2,748
3,0,0,0,0,0,0,3,286
4,0,0,0,0,0,0,4,155
...,...,...,...,...,...,...,...,...
82021,872,870,166,357,187,197,536,1
82022,876,837,168,273,126,148,498,1
82023,914,930,182,201,40,109,553,1
82024,931,974,180,349,146,126,519,1


In [ ]:
fig = go.Figure(
    data=[go.Bar(x=copiadados['data_inicio'], y=[1, 3, 2])],
    layout=dict(title=dict(text="A Figure Specified By A Graph Object"))
)


copiadados.groupby("Questões administrativas / funcionamento")["Ibge"].count()


fig = go.Figure()
fig.update_layout(title=("Evolução dos tipos de reuniões por Tempo"))
fig.add_trace(go.Bar(x=copiadados['data_inicio'],y=dadosagrupadosportema['Questões administrativas / funcionamento'], name='Questões administrativas / funcionamento'))
# fig.add_trace(go.Bar(x=dadosagrupadosportema['data_inicio'],y=dadosagrupadosportema['Processos de trabalho'], name='Processos de trabalho'))
# fig.add_trace(go.Bar(x=dadosagrupadosportema['data_inicio'],y=dadosagrupadosportema['Diagnóstico do território / monitoramento do terrotório'], name='Diagnóstico do território / monitoramento do terrotório'))
# fig.add_trace(go.Bar(x=dadosagrupadosportema['data_inicio'],y=dadosagrupadosportema['Planejamento / monitoramento das ações da equipe'], name='Planejamento / monitoramento das ações da equipe'))
# fig.add_trace(go.Bar(x=dadosagrupadosportema['data_inicio'],y=dadosagrupadosportema['Discussão de caso / projeto terapêutico singular'], name='Discussão de caso / projeto terapêutico singular'))
# fig.add_trace(go.Bar(x=dadosagrupadosportema['data_inicio'],y=dadosagrupadosportema['Educação permanente'], name='Educação permanente'))
# fig.add_trace(go.Bar(x=dadosagrupadosportema['data_inicio'],y=dadosagrupadosportema['Outros'], name='Outros'))
fig.show()

### Evolução pelo tempo

In [6]:
copiadados = dadosrelativcoletiva

In [7]:
dadosagrupadosportema = copiadados.groupby(['data_inicio'], as_index=False)['Questões administrativas / funcionamento', 'Processos de trabalho',
       'Diagnóstico do território / monitoramento do terrotório', 'Planejamento / monitoramento das ações da equipe',
       'Discussão de caso / projeto terapêutico singular', 'Educação permanente', 'Outros'].sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [8]:
fig = go.Figure()
fig.update_layout(title=("Evolução dos tipos de reuniões por Tempo"))
fig.add_trace(go.Scatter(x=dadosagrupadosportema['data_inicio'],y=dadosagrupadosportema['Questões administrativas / funcionamento'], mode='markers+lines', name='Questões administrativas / funcionamento'))
fig.add_trace(go.Scatter(x=dadosagrupadosportema['data_inicio'],y=dadosagrupadosportema['Processos de trabalho'], mode='markers+lines', name='Processos de trabalho'))
fig.add_trace(go.Scatter(x=dadosagrupadosportema['data_inicio'],y=dadosagrupadosportema['Diagnóstico do território / monitoramento do terrotório'], mode='markers+lines', name='Diagnóstico do território / monitoramento do terrotório'))
fig.add_trace(go.Scatter(x=dadosagrupadosportema['data_inicio'],y=dadosagrupadosportema['Planejamento / monitoramento das ações da equipe'], mode='markers+lines', name='Planejamento / monitoramento das ações da equipe'))
fig.add_trace(go.Scatter(x=dadosagrupadosportema['data_inicio'],y=dadosagrupadosportema['Discussão de caso / projeto terapêutico singular'], mode='markers+lines', name='Discussão de caso / projeto terapêutico singular'))
fig.add_trace(go.Scatter(x=dadosagrupadosportema['data_inicio'],y=dadosagrupadosportema['Educação permanente'], mode='markers+lines', name='Educação permanente'))
fig.add_trace(go.Scatter(x=dadosagrupadosportema['data_inicio'],y=dadosagrupadosportema['Outros'], mode='markers+lines', name='Outros'))
fig.show()

### Variação dos tipos de reuniões nos municipios

In [ ]:
copiadados

In [ ]:
fig = go.Figure(
    data=[go.Bar(x=[1, 2, 3], y=[1, 3, 2])],
    layout=dict(title=dict(text="A Figure Specified By A Graph Object"))
)

fig.show()